## week08: Text classification with simple features

In [ ]:
import heapq
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import StratifiedKFold

%matplotlib inline

# Задача классификации текстов

Задача классификации текстов заключается в том, чтобы определить по документу его класс.
В данном случае предлагается рассмотреть в качестве документов - письма, заранее отклассифицированных по 20 темам.

In [ ]:
all_categories = fetch_20newsgroups().target_names
all_categories

Возьмём всего 3 темы, но из одного раздела (документы из близких тем сложнее отличать друг от друга)

In [ ]:
categories = [
    'sci.electronics',
    'sci.space',
    'sci.med'
]

train_data = fetch_20newsgroups(subset='train',
                                categories=categories,
                                remove=('headers', 'footers', 'quotes'))

test_data = fetch_20newsgroups(subset='test',
                               categories=categories,
                               remove=('headers', 'footers', 'quotes'))

## Векторизация текстов
**Вопрос: как описать текстовые документы пространством признаков?**


**Идея №1**: мешок слов (bag-of-words) - каждый документ или текст выглядит как неупорядоченный набор слов без сведений о связях между ними.
<img src='https://st2.depositphotos.com/2454953/9959/i/450/depositphotos_99593622-stock-photo-holidays-travel-bag-word-cloud.jpg'>

**Идея №2**: создаём вектор "слов", каждая компонента отвечает отдельному слову.

Для векторизации текстов воспользуемся [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Можно всячески варировать извлечение признаков (убирать редкие слова, убирать частые слова, убирать слова общей лексики, брать биграмы и т.д.)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
CountVectorizer()

In [ ]:
count_vectorizer = CountVectorizer(min_df=5, ngram_range=(1, 2)) 

In [ ]:
sparse_feature_matrix = count_vectorizer.fit_transform(train_data.data)
sparse_feature_matrix

In [ ]:
num_2_words = {
    v: k
    for k, v in count_vectorizer.vocabulary_.items()
}

Слова с наибольшим положительным весом, являются характерными словами темы

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV

Воспользуемся `macro`-average для оценки качества решения в задаче многоклассовой классификации.

In [ ]:
f_scorer = make_scorer(f1_score, average='macro')

Обучим логистическую регрессию для предсказания темы документа

In [ ]:
algo = LogisticRegression(C=0.00001)
algo.fit(sparse_feature_matrix, train_data.target)

In [ ]:
W = algo.coef_.shape[1]
for c in algo.classes_:
    topic_words = [
        num_2_words[w_num]
        for w_num in heapq.nlargest(10, range(W), key=lambda w: algo.coef_[c, w])
    ]
    print(',  '.join(topic_words))


Сравним качество на обучающей и отложенной выборках.

In [ ]:
algo.fit(sparse_feature_matrix, train_data.target)

In [ ]:
f_scorer(algo, sparse_feature_matrix, train_data.target)

In [ ]:
f_scorer(algo, count_vectorizer.transform(test_data.data), test_data.target)

Значения f-меры получились очень низкие.

**Вопрос:** в чём причина?

In [ ]:
plt.hist(algo.coef_[0], bins=500)
plt.xlim([-0.0006, 0.0006])
plt.show()

** Какую выбрать метрику для регуляризации? **

In [ ]:
algo = LogisticRegression(penalty='l1', C=0.1)
arr = cross_val_score(algo, sparse_feature_matrix, train_data.target, cv=5, scoring=f_scorer)
print(arr)
print(np.mean(arr))

In [ ]:
algo.fit(sparse_feature_matrix, train_data.target)

In [ ]:
f_scorer(algo, sparse_feature_matrix, train_data.target)

In [ ]:
f_scorer(algo, count_vectorizer.transform(test_data.data), test_data.target)

Подберём оптимальное значение параметра регуляризации

In [ ]:
def grid_plot(x, y, x_label, title, y_label='f_measure'):
    plt.figure(figsize=(12, 6))
    plt.grid(True),
    plt.plot(x, y, 'go-')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

In [ ]:
print(*map(float, np.logspace(-2, 2, 10)))

In [ ]:
lr_grid = {
    'C': np.logspace(-2, 2, 10),
}
gs = GridSearchCV(LogisticRegression(penalty='l1'), lr_grid, scoring=f_scorer, cv=5, n_jobs=5)
%time  gs.fit(sparse_feature_matrix, train_data.target)
print("best_params: {}, best_score: {}".format(gs.best_params_, gs.best_score_))

Рассмотрим график:

In [ ]:
grid_plot(
    lr_grid['C'], gs.cv_results_['mean_test_score'], 'C - coefficient of regularization', 'LogReg(penalty=l1)'
)

In [ ]:
lr_grid = {
    'C': np.linspace(1, 20, 40),
}
gs = GridSearchCV(LogisticRegression(penalty='l1'), lr_grid, scoring=f_scorer, cv=5, n_jobs=5)
%time  gs.fit(sparse_feature_matrix, train_data.target)
print("best_params: {}, best_score: {}".format(gs.best_params_, gs.best_score_))

In [ ]:
grid_plot(
    lr_grid['C'], gs.cv_results_['mean_test_score'], 'C - coefficient of regularization', 'LogReg(penalty=l1)'
)

In [ ]:
lr_final = LogisticRegression(penalty='l1', C=10)
%time lr_final.fit(sparse_feature_matrix, train_data.target)

In [ ]:
accuracy_score(lr_final.predict(sparse_feature_matrix), train_data.target)

In [ ]:
f_scorer(lr_final, sparse_feature_matrix, train_data.target)

In [ ]:
accuracy_score(lr_final.predict(count_vectorizer.transform(test_data.data)), test_data.target)

In [ ]:
f_scorer(lr_final, count_vectorizer.transform(test_data.data), test_data.target)

## Регуляризация вместе с векторизацией признаков
Чтобы не делать векторизацию и обучение раздельно, есть удобный класс Pipeline. Он позволяет объединить в цепочку последовательность действий

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ("vectorizer", CountVectorizer(min_df=5, ngram_range=(1, 2))),
    ("algo", LogisticRegression())
])

In [ ]:
pipeline.fit(train_data.data, train_data.target)

In [ ]:
f_scorer(pipeline, train_data.data, train_data.target)

In [ ]:
f_scorer(pipeline, test_data.data, test_data.target)

Значения такие же как мы получали ранее, делая шаги раздельно.

In [ ]:
from sklearn.pipeline import make_pipeline

При кроссвалидации нужно, чтобы CountVectorizer не обучался на тесте (иначе объекты становятся зависимыми). Pipeline позволяет это просто сделать.

In [ ]:
pipeline = make_pipeline(CountVectorizer(min_df=5, ngram_range=(1, 2)), LogisticRegression())
arr = cross_val_score(pipeline, train_data.data, train_data.target, cv=5, scoring=f_scorer)
print(arr)
print(np.mean(arr))

В Pipeline можно добавлять новые шаги препроцессинга данных

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
pipeline = make_pipeline(CountVectorizer(min_df=5, ngram_range=(1, 2)), TfidfTransformer(), LogisticRegression())
arr = cross_val_score(pipeline, train_data.data, train_data.target, cv=5, scoring=f_scorer)
print(arr)
print(np.mean(arr))

In [ ]:
pipeline.fit(train_data.data, train_data.target)

In [ ]:
accuracy_score(pipeline.predict(train_data.data), train_data.target)

In [ ]:
f_scorer(pipeline, train_data.data, train_data.target)

In [ ]:
accuracy_score(pipeline.predict(test_data.data), test_data.target)

In [ ]:
f_scorer(pipeline, test_data.data, test_data.target)

Качество стало немного лучше

# Классификация сообщений чатов

В качестве задания предлагается построить модель классификации текстов, соответствующих сообщениям из чатов по ML, Python и знакомствам.

**Данные** можно взять с <a src="https://www.kaggle.com/c/tfstextclassification">соревнования на Kaggle</a>, проведенное в рамках курса "Диалоговые системы" в Тинькофф. Прямая [ссылка](https://www.dropbox.com/s/8wckwzfy63ajxpm/tfstextclassification.zip?dl=0) на скачивание.

In [ ]:
import pandas as pd

In [ ]:
# data_path = 'data/{}'
df = pd.read_csv('data/train.csv')

### Первичный анализ данных

In [ ]:
print(df.shape)

In [ ]:
df.head()

In [ ]:
label = 0
print('Label: ', label, '\n'+'='*100+'\n')
print(*df[df['label'] == label].sample(10).text, sep='\n'+'-'*100+'\n\n')

In [ ]:
label = 1
print('Label: ', label, '\n'+'='*100+'\n')
print(*df[df['label'] == label].sample(10).text, sep='\n'+'-'*100+'\n\n')

In [ ]:
label = 2
print('Label: ', label, '\n'+'='*100+'\n')
print(*df[df['label'] == label].sample(10).text, sep='\n'+'-'*100+'\n\n')

### Разделим данные на train/test

In [ ]:
skf = StratifiedKFold(3, random_state=37)
train_index, test_index = next(skf.split(df.text, df.label))
train_df, test_df = df.iloc[train_index], df.iloc[test_index]
print(train_df.shape, test_df.shape)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## Baseline

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from xgboost.sklearn import XGBClassifier

Преобразуем данные

In [ ]:
X_train = train_df.text
y_train = train_df.label
print(X_train.shape)

X_test = test_df.text
y_test = test_df.label
print(X_test.shape)

Подготовим pipeline

In [ ]:
pipeline = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("clf", DecisionTreeClassifier()),
])

Обучим классификатор

In [ ]:
%%time
clf = pipeline
clf.fit(X_train, y_train)

Оценим качество

In [ ]:
print("Train_acc: {:.4f}, train_f-measure: {:.4f}".format(
    accuracy_score(clf.predict(X_train), y_train),
    f_scorer(clf, X_train, y_train)
))

In [ ]:
print("Test_acc: {:.4f}, test_f-measure: {:.4f}".format(
    accuracy_score(clf.predict(X_test), y_test),
    f_scorer(clf, X_test, y_test)
))

### Your turn

Как видим, наша модель переобучилась. Для получения лучших результатов попробуйте воспользоваться более хитрыми и походящими инструментами.

1. Попробуйте поработать с параметрами `CountVectorizer`.
2. Попробуйте воспользоваться TF-IDF для кодирования текстовой информации ([ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)).
3. Попробуйте воспользоваться другими моделями и средствами снижения размерности.

Формальный критерий успешности выполнения данного (опционального) задания: 
* Проведен честный эксперимент с апробацией различных методов (>=3)
* Полученный алгоритм не выказывает явных следов переобучения (качество на train и test не различаются более, чем на 0.03 условных попугая)
* Test accuracy >= 0.835, f1-score >= 0.815